# Instagram Post Scraper

In [141]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import re

In [121]:
browser  = webdriver.Firefox()
browser.get('https://www.instagram.com/')

In [122]:
WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='username']")))
username = browser.find_element(By.CSS_SELECTOR, "input[name='username']")
password = browser.find_element(By.CSS_SELECTOR, "input[name='password']")
username.clear()
password.clear()
username.send_keys("ithinktheytoldthewarden")
password.send_keys("lndlndlnd")

In [123]:
time.sleep(5)
login_button = browser.find_element(By.CSS_SELECTOR, "button[type='submit']")
login_button.click()

In [124]:
time.sleep(5)

In [125]:
username='donttellthewarden'
browser.get('https://www.instagram.com/'+username+'/?hl=en')
links = set()

In [126]:
n_scrolls = 20
for _ in range(n_scrolls):
    source = browser.page_source
    data=bs(source, 'html.parser')
    posts = data.find_all('a', class_='_a6hd')
    time.sleep(15)
    
    for post in posts:
        href = post.get('href')
        if href.startswith('/p/') or href.startswith('/reels/'):
            link = "https://instagram.com" + href
            links.add(link)
            
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [127]:
print(links)

{'https://instagram.com/p/CVxmEKAPLKo/?hl=en', 'https://instagram.com/p/C7mE2Yxv5JS/?hl=en', 'https://instagram.com/p/CQqvOgCMUye/?hl=en', 'https://instagram.com/p/CVsP373vkMm/?hl=en', 'https://instagram.com/p/Ct9Zmodsk-1/?hl=en', 'https://instagram.com/p/C2KM8Vov7-N/?hl=en', 'https://instagram.com/p/CO7gMuAjAPz/?hl=en', 'https://instagram.com/p/C4aqJIdvE1N/?hl=en', 'https://instagram.com/p/CsqtpaWvBGF/?hl=en', 'https://instagram.com/p/CbkHMknPQO6/?hl=en', 'https://instagram.com/p/CU5JflxPqxO/?hl=en', 'https://instagram.com/p/Cjx4RuLPdkQ/?hl=en', 'https://instagram.com/p/CXtcYmrvGse/?hl=en', 'https://instagram.com/p/CPx8GCsDTJy/?hl=en', 'https://instagram.com/p/Ct__UPEPFEs/?hl=en', 'https://instagram.com/p/CfjLiNmBDzc/?hl=en', 'https://instagram.com/p/CO2ESyvjT-n/?hl=en', 'https://instagram.com/p/C7HcEfVPwA4/?hl=en', 'https://instagram.com/p/CPBQx_AjtRn/?hl=en', 'https://instagram.com/p/CUILXslv4OQ/?hl=en', 'https://instagram.com/p/Cty-5aXvIrC/?hl=en', 'https://instagram.com/p/C5TWSZ1v

In [128]:
print(len(links))

115


In [142]:
data_list = []

for i, link in enumerate(links):
    # if i >= 20:
        # break

    browser.get(link)
    time.sleep(5)

    source = browser.page_source
    data=bs(source, 'html.parser')
    
    time_elements = data.find_all('time', class_='x1p4m5qa')
    
    for time_element in time_elements:
        datetime_attr = time_element.get('datetime')
        title_attr = time_element.get('title')
        
        dt_object = datetime.fromisoformat(datetime_attr.rstrip('Z'))
        date_of_posting = dt_object.date()
        time_of_posting = dt_object.time()
        
        time_in_IST = (datetime.combine(date_of_posting, time_of_posting) + timedelta(hours=4, minutes=30)).time()
            
        likes_element = data.find('span', class_='html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs')
        likes = likes_element.text if likes_element else '0'  
        
        data_list.append({
                'title': title_attr,
                'date_of_posting': date_of_posting,
                'time_of_posting': time_of_posting,
                'time_in_IST': time_in_IST,
                'likes': likes
            })

In [143]:
print(data_list)

[{'title': 'Nov 2, 2021', 'date_of_posting': datetime.date(2021, 11, 2), 'time_of_posting': datetime.time(13, 20, 9), 'time_in_IST': datetime.time(17, 50, 9), 'likes': '41'}, {'title': 'May 30, 2024', 'date_of_posting': datetime.date(2024, 5, 30), 'time_of_posting': datetime.time(13, 51, 15), 'time_in_IST': datetime.time(18, 21, 15), 'likes': '95'}, {'title': 'Jun 28, 2021', 'date_of_posting': datetime.date(2021, 6, 28), 'time_of_posting': datetime.time(14, 48, 33), 'time_in_IST': datetime.time(19, 18, 33), 'likes': '168'}, {'title': 'Oct 31, 2021', 'date_of_posting': datetime.date(2021, 10, 31), 'time_of_posting': datetime.time(11, 30, 2), 'time_in_IST': datetime.time(16, 0, 2), 'likes': '160'}, {'title': 'Jun 26, 2023', 'date_of_posting': datetime.date(2023, 6, 26), 'time_of_posting': datetime.time(15, 55, 36), 'time_in_IST': datetime.time(20, 25, 36), 'likes': '131'}, {'title': 'Jan 16, 2024', 'date_of_posting': datetime.date(2024, 1, 16), 'time_of_posting': datetime.time(11, 26, 15

# Data Analysis

In [ ]:
import pandas as pd

In [144]:
df = pd.DataFrame(data_list)
df.set_index('title', inplace=True)
df.sort_values(by='date_of_posting', ascending=False, inplace=True)

In [145]:
df

,date_of_posting,time_of_posting,time_in_IST,likes
title,,,,
"May 30, 2024",2024-05-30,13:51:15,18:21:15,95
"May 18, 2024",2024-05-18,16:17:41,20:47:41,143
"Mar 12, 2024",2024-03-12,13:52:03,18:22:03,189
"Jan 16, 2024",2024-01-16,11:26:15,15:56:15,176
"Jun 27, 2023",2023-06-27,16:03:37,20:33:37,127
"Jun 26, 2023",2023-06-26,15:55:36,20:25:36,131
"May 25, 2023",2023-05-25,13:13:47,17:43:47,189
"Oct 16, 2022",2022-10-16,15:18:48,19:48:48,261
"Jul 3, 2022",2022-07-03,12:12:54,16:42:54,144
